# OmniSciDB set execution mode

In [1]:
from copy import copy
from typing import Optional

import ibis
import pandas as pd
import pymapd
# local
from settings import conf
from utils import cursor2df

### Setup

In [2]:
con = ibis.omniscidb.connect(**conf['omniscidb'])
con

### PyMapD set_execution_mode

In [3]:
t = con.table('functional_alltypes')
sql = t.head(1).compile()
sql

'SELECT *\nFROM functional_alltypes\nLIMIT 1'

In [4]:
cursor2df(con.con.execute(sql))

,index,Unnamed__0,id,bool_col,tinyint_col,smallint_col,int_col,bigint_col,float_col,double_col,date_string_col,string_col,timestamp_col,year_,month_
0,0,0,6690,1,0,0,0,0,0.0,0.0,11/01/10,0,2010-11-01,2010,11


In [5]:
con.con._session

'Z3bqpR1SJuOXn51l1ei03gmfHpX8K3Mp'

In [6]:
con.con._client.set_execution_mode(con.con._session, 2)


### Ibis execution_mode implementation

In this prototype, `ibis.omniscidb.client.OmniSciDBClient._execute`, 
the method responsible for the prepare the **SQL** execution, is wrapped 
to demonstrate the changes needed by `set_execution_mode` implementation.

In [7]:
con.execution_mode('cpu')
con.execute(t.head())

,index,Unnamed__0,id,bool_col,tinyint_col,smallint_col,int_col,bigint_col,float_col,double_col,date_string_col,string_col,timestamp_col,year_,month_
0,0,0,6690,True,0,0,0,0,0.0,0.0,11/01/10,0,2010-11-01 00:00:00,2010,11
1,1,1,6691,False,1,1,1,10,1.1,10.1,11/01/10,1,2010-11-01 00:01:00,2010,11
2,2,2,6692,True,2,2,2,20,2.2,20.2,11/01/10,2,2010-11-01 00:02:00,2010,11
3,3,3,6693,False,3,3,3,30,3.3,30.3,11/01/10,3,2010-11-01 00:03:00,2010,11
4,4,4,6694,True,4,4,4,40,4.4,40.4,11/01/10,4,2010-11-01 00:04:00,2010,11


In [8]:
try:
    con.execution_mode('gpu')
except Exception as e:
    assert e.error_msg == 'Cannot switch to GPU mode in a server started in CPU-only mode.'